# **스팀 리뷰 감정분석 with Hugging Face BERT**



<br>
<br>

# **준비 사항**

In [1]:
# Hugging Face의 트랜스포머 모델을 설치
!pip install transformers

     |████████████████████████████████| 1.5MB 8.6MB/s 
     |████████████████████████████████| 2.9MB 38.4MB/s 
     |████████████████████████████████| 890kB 43.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=c99ca433e3e2ca016a2f1c44821da91dbcb78737f7c6add2a5099e98f0a0ede9
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

<br>
<br>

# **데이터 로드**

In [4]:
# 스팀 리뷰 감정분석 데이터 다운로드
# !git clone https://github.com/bab2min/corpus

In [5]:
# %cd /content/corpus/sentiment

In [6]:
# 디렉토리의 파일 목록
# !ls

In [7]:
# 판다스로 데이터 불러오기
df = pd.read_csv("/content/drive/MyDrive/DataScience/data/corpus/sentiment/steam.txt", sep='\t',
                 names=['label', 'document'])

print(df.shape)

(100000, 2)


In [8]:
# 결측값 확인
df.isna().sum()

label       0
document    0
dtype: int64

In [9]:
df

,label,document
0,0,노래가 너무 적음
1,0,"돌겠네 진짜. 황숙아, 어크 공장 그만 돌려라. 죽는다."
2,1,막노동 체험판 막노동 하는사람인데 장비를 내가 사야돼 뭐지
3,1,차악!차악!!차악!!! 정말 이래서 왕국을 되찾을 수 있는거야??
4,1,시간 때우기에 좋음.. 도전과제는 50시간이면 다 깰 수 있어요
...,...,...
99995,0,한글화해주면 10개산다
99996,0,개쌉노잼 ㅋㅋ
99997,0,노잼이네요... 30분하고 지웠어요...
99998,1,야생을 사랑하는 사람들을 위한 짧지만 여운이 남는 이야기. 영어는 그리 어렵지 않습니다.


In [10]:
df['label'].value_counts(normalize=True)

0    0.50004
1    0.49996
Name: label, dtype: float64

In [11]:
# 훈련셋과 검증셋으로 분리
train, test = train_test_split(df,
                               random_state=3, 
                               test_size=0.25,
                               stratify=df['label'])

print(f"학습용데이터 : {train.shape}\n테스트용데이터 : {test.shape}")

학습용데이터 : (75000, 2)
테스트용데이터 : (25000, 2)


In [12]:
# 데이터별 타겟 분포 확인
print(train['label'].value_counts(normalize=True))
print(test['label'].value_counts(normalize=True))

0    0.50004
1    0.49996
Name: label, dtype: float64
0    0.50004
1    0.49996
Name: label, dtype: float64


In [13]:
# 훈련셋의 앞부분 출력
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)
train.head(10)

,label,document
0,0,이걸 돈내고 사준놈이 제일 이상한놈임
1,1,찍기로 깨지 마세요 중국인 장루원들 이름도 다 추리해서 푸는 겁니다 저처럼 맞을 때...
2,1,정말 환상적인 그래픽에 푹 빠져서 게임을 했습니다. 퍼즐은 쉬운편인데 화면이 너무 ...
3,0,아주 훌륭한 소프트웨어다! 창작마당만 빼고...
4,1,좋다. 제목에 액션이 있긴 하지만 실제론 셰이커 흔드는 정도밖에 없으니 액션을 기대...
5,1,2만원 가격값 못합니다. 하지만 이것은 RWBY 팬이 매우 높게 평가
6,0,팔고 스타듀밸리 사고 싶다.
7,1,이 게임은 당신의 창의성을 만들어 아주 좋은 게임입니다. 자신이 만들고 싶은 우주선...
8,1,★★★★★ 말이필요없다
9,0,눈이 즐겁고 이쁜 게임 그러나 그게 전부


<br>
<br>

# **전처리 - 훈련셋**

In [14]:
# 리뷰 문장 추출
sentences = train['document']
sentences[:10]

0                                 이걸 돈내고 사준놈이 제일 이상한놈임
1    찍기로 깨지 마세요 중국인 장루원들 이름도 다 추리해서 푸는 겁니다 저처럼 맞을 때...
2    정말 환상적인 그래픽에 푹 빠져서 게임을 했습니다. 퍼즐은 쉬운편인데 화면이 너무 ...
3                           아주 훌륭한 소프트웨어다! 창작마당만 빼고...
4    좋다. 제목에 액션이 있긴 하지만 실제론 셰이커 흔드는 정도밖에 없으니 액션을 기대...
5               2만원 가격값 못합니다. 하지만 이것은 RWBY 팬이 매우 높게 평가
6                                      팔고 스타듀밸리 사고 싶다.
7    이 게임은 당신의 창의성을 만들어 아주 좋은 게임입니다. 자신이 만들고 싶은 우주선...
8                                         ★★★★★ 말이필요없다
9                               눈이 즐겁고 이쁜 게임 그러나 그게 전부
Name: document, dtype: object

In [15]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 이걸 돈내고 사준놈이 제일 이상한놈임 [SEP]',
 '[CLS] 찍기로 깨지 마세요 중국인 장루원들 이름도 다 추리해서 푸는 겁니다 저처럼 맞을 때까지 아무 이름이나 적어넣어서 찍는 방식으로 깨면 이번엔 제대로 깰 테니 제발 뇌 리셋해달라고 울부짖게 됩니다 시스템 이용해서 편법으로 깨지 말고 정공법으로 푸십쇼 [SEP]',
 '[CLS] 정말 환상적인 그래픽에 푹 빠져서 게임을 했습니다. 퍼즐은 쉬운편인데 화면이 너무 어두운 구간에서는 진행방향을 알기 힘듭니다. 약간 게임플레이에 허술한 면이 있는 것을 빼면 상당히 괜찮은 게임입니다. [SEP]',
 '[CLS] 아주 훌륭한 소프트웨어다! 창작마당만 빼고... [SEP]',
 '[CLS] 좋다. 제목에 액션이 있긴 하지만 실제론 셰이커 흔드는 정도밖에 없으니 액션을 기대하지는 말자. 그래도 좋다. [SEP]',
 '[CLS] 2만원 가격값 못합니다. 하지만 이것은 RWBY 팬이 매우 높게 평가 [SEP]',
 '[CLS] 팔고 스타듀밸리 사고 싶다. [SEP]',
 '[CLS] 이 게임은 당신의 창의성을 만들어 아주 좋은 게임입니다. 자신이 만들고 싶은 우주선을 마음껏 만들 수 있어 매우 좋은 게임 같습니다. [SEP]',
 '[CLS] ★★★★★ 말이필요없다 [SEP]',
 '[CLS] 눈이 즐겁고 이쁜 게임 그러나 그게 전부 [SEP]']

![대체 텍스트](https://mino-park7.github.io/images/2019/02/bert-input-representation.png)

BERT의 입력은 위의 그림과 같은 형식입니다. Classification을 뜻하는 [CLS] 심볼이 제일 앞에 삽입됩니다. 파인튜닝시 출력에서 이 위치의 값을 사용하여 분류를 합니다. [SEP]은 Seperation을 가리키는데, 두 문장을 구분하는 역할을 합니다. 이 예제에서는 문장이 하나이므로 [SEP]도 하나만 넣습니다.
<br>
<br>
<br>

In [16]:
# 라벨 추출
labels = train['label'].values
labels

array([0, 1, 1, ..., 0, 0, 0])

In [17]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])


[CLS] 이걸 돈내고 사준놈이 제일 이상한놈임 [SEP]
['[CLS]', '이', '##걸', '돈', '##내', '##고', '사', '##준', '##놈', '##이', '제', '##일', '이상', '##한', '##놈', '##임', '[SEP]']


BERT는 형태소분석으로 토큰을 분리하지 않습니다. WordPiece라는 통계적인 방식을 사용합니다. 한 단어내에서 자주 나오는 글자들을 붙여서 하나의 토큰으로 만듭니다. 이렇게 하면 언어에 상관없이 토큰을 생성할 수 있다는 장점이 있습니다. 또한 신조어 같이 사전에 없는 단어를 처리하기도 좋습니다. 

위의 결과에서 ## 기호는 앞 토큰과 이어진다는 표시입니다. 토크나이저는 여러 언어의 데이터를 기반으로 만든 'bert-base-multilingual-cased'를 사용합니다. 그래서 한글도 처리가 가능합니다.
<br>
<br>
<br>

In [18]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   9638, 118624,   9089,  31605,  11664,   9405,  54867,
       118745,  10739,   9672,  18392,  66982,  11102, 118745,  36240,
          102,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

보통 딥러닝 모델에는 토큰 자체를 입력으로 넣을 수 없습니다. 임베딩 레이어에는 토큰을 숫자로 된 인덱스로 변환하여 사용합니다. BERT의 토크나이저는 {단어토큰:인덱스}로 구성된 단어사전을 가지고 있습니다. 이를 참조하여 토큰을 인덱스로 바꿔줍니다.
<br>
<br>
<br>

In [19]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [20]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2018, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([  101,   100,  9838, 11664,   100,   102,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [21]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

<br>
<br>

# **전처리 - 테스트셋**

In [22]:
# 리뷰 문장 추출
sentences = test['document']
sentences[:10]

0    최적화 발로했습니다. 권장사양 믿지 마세요. 4790에 1060 쓰는데 중옵으로 해...
1    이거 세이브파일 서버에서 저장이 안되나보내요 짜증나내요 8시간 한거 날라가니까. 동...
2                    Dirt3 = GFWL 돈주고 샀는데 게임을 못하다니 ㅋㅋㅋ
3    식인종 처죽여서 먹고 식인종 조지고 그러다 모험이라도 떠날라면 난이도 개같이 책정해...
4                                           모자걸이 너모커엽다
5                           곧 소돔이 추가되며 완벽한 갓겜이 될 예정이자너
6    가끔 개 짜증나는 구간이 있기는 한데 타격감도 좋고 재밌습니다. 근데 세이브는 아무...
7    자기얼굴에 맞춰서 센서 감도를 설정할수잇어요! 시간만좀 들이면 더 재밋어짐! 그리구...
8    Are you kidding me? There are a lot of mouse p...
9          고기 다지는게 이렇게 재밌는줄 몰랐습니다 이제부터 저의 꿈은 정육사 입니다^^
Name: document, dtype: object

In [23]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 최적화 발로했습니다. 권장사양 믿지 마세요. 4790에 1060 쓰는데 중옵으로 해도 프레임이 30~40에서 놉니다. 자잘한 버그도 많구요. 역시 유비겜은 거르는게 답 그리고 해상도를 분명 1080p로 맞춰놨는데도 미친듯한 계단현상이 일어납니다. [SEP]',
 '[CLS] 이거 세이브파일 서버에서 저장이 안되나보내요 짜증나내요 8시간 한거 날라가니까. 동영상 스킵도 안되고 대사 일일이 다봐야되고 너무 답답하내요 [SEP]',
 '[CLS] Dirt3 = GFWL 돈주고 샀는데 게임을 못하다니 ㅋㅋㅋ [SEP]',
 '[CLS] 식인종 처죽여서 먹고 식인종 조지고 그러다 모험이라도 떠날라면 난이도 개같이 책정해서 암것도 못함 병신겜 [SEP]',
 '[CLS] 모자걸이 너모커엽다 [SEP]',
 '[CLS] 곧 소돔이 추가되며 완벽한 갓겜이 될 예정이자너 [SEP]',
 '[CLS] 가끔 개 짜증나는 구간이 있기는 한데 타격감도 좋고 재밌습니다. 근데 세이브는 아무 데서나 하게 해주면 안 되냐.. [SEP]',
 '[CLS] 자기얼굴에 맞춰서 센서 감도를 설정할수잇어요! 시간만좀 들이면 더 재밋어짐! 그리구 친구랑 스카이프할때 꿀잼!! 이건꼭 스팀워크샾이 나와야함... [SEP]',
 '[CLS] Are you kidding me? There are a lot of mouse pointer errors. 시발 장난해? 마우스포인터 오류 존나 많아서 열쇠 따지도 못하고 [SEP]',
 '[CLS] 고기 다지는게 이렇게 재밌는줄 몰랐습니다 이제부터 저의 꿈은 정육사 입니다^^ [SEP]']

In [24]:
# 라벨 추출
labels = test['label'].values
labels

array([0, 0, 0, ..., 0, 1, 1])

In [25]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] 최적화 발로했습니다. 권장사양 믿지 마세요. 4790에 1060 쓰는데 중옵으로 해도 프레임이 30~40에서 놉니다. 자잘한 버그도 많구요. 역시 유비겜은 거르는게 답 그리고 해상도를 분명 1080p로 맞춰놨는데도 미친듯한 계단현상이 일어납니다. [SEP]
['[CLS]', '최', '##적', '##화', '발', '##로', '##했', '##습', '##니다', '.', '권', '##장', '##사', '##양', '믿', '##지', '마', '##세', '##요', '.', '479', '##0', '##에', '1060', '쓰', '##는데', '중', '##옵', '##으로', '해', '##도', '프', '##레', '##임', '##이', '30', '~', '40', '##에서', '[UNK]', '.', '자', '##잘', '##한', '버', '##그', '##도', '많', '##구', '##요', '.', '역시', '[UNK]', '거', '##르는', '##게', '답', '그리고', '해', '##상', '##도를', '분', '##명', '1080', '##p', '##로', '[UNK]', '미', '##친', '##듯', '##한', '계', '##단', '##현', '##상이', '일', '##어', '##납', '##니다', '.', '[SEP]']


In [26]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   9764,  14801,  18227,   9323,  11261, 119424, 119081,
        48345,    119,   8917,  13890,  12945,  37114,   9312,  12508,
         9246,  24982,  48549,    119,  44822,  10929,  10530,  94142,
         9511,  41850,   9694, 119157,  11467,   9960,  12092,   9942,
        56645,  36240,  10739,  10244,    198,  10533,  11489,    100,
          119,   9651, 119197,  11102,   9336,  78136,  12092,   9249,
        17196,  48549,    119,  58466,    100,   8863,  47908,  14153,
         9065,  23289,   9960,  14871,  52602,   9367,  16758,  47549,
        10410,  11261,    100,   9309,  55358, 118817,  11102,   8887,
        24989,  30842,  83902,   9641,  12965, 118716,  48345,    119,
          102,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [27]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [28]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([   101,   9764,  14801,  18227,   9323,  11261, 119424, 119081,  48345,
           119,   8917,  13890,  12945,  37114,   9312,  12508,   9246,  24982,
         48549,    119,  44822,  10929,  10530,  94142,   9511,  41850,   9694,
        119157,  11467,   9960,  12092,   9942,  56645,  36240,  10739,  10244,
           198,  10533,  11489,    100,    119,   9651, 119197,  11102,   9336,
         78136,  12092,   9249,  17196,  48549,    119,  58466,    100,   8863,
         47908,  14153,   9065,  23289,   9960,  14871,  52602,   9367,  16758,
         47549,  10410,  11261,    100,   9309,  55358, 118817,  11102,   8887,
         24989,  30842,  83902,   9641,  12965, 118716,  48345,    119,    102,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [29]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

<br>
<br>

# **모델 생성**

In [30]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [31]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [32]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

![대체 텍스트](http://www.mccormickml.com/assets/BERT/padding_and_mask.png)

사전훈련된 BERT는 다양한 문제로 전이학습이 가능합니다. 여기서는 위의 그림과 같이 한 문장을 분류하는 방법을 사용합니다. 영화리뷰 문장이 입력으로 들어가면, 긍정/부정으로 구분합니다. 모델의 출력에서 [CLS] 위치인 첫 번째 토큰에 새로운 레이어를 붙여서 파인튜닝을 합니다. Huggning Face는 BertForSequenceClassification() 함수를 제공하기 때문에 쉽게 구현할 수 있습니다.
<br>
<br>
<br>

In [33]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

<br>
<br>

# **모델 학습**

In [34]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [35]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [36]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch   500  of  2,110.    Elapsed: 0:03:28.
  Batch 1,000  of  2,110.    Elapsed: 0:06:55.
  Batch 1,500  of  2,110.    Elapsed: 0:10:22.
  Batch 2,000  of  2,110.    Elapsed: 0:13:50.

  Average training loss: 0.51
  Training epcoh took: 0:14:35

Running Validation...
  Accuracy: 0.78
  Validation took: 0:00:29

======== Epoch 2 / 4 ========
Training...
  Batch   500  of  2,110.    Elapsed: 0:03:27.
  Batch 1,000  of  2,110.    Elapsed: 0:06:55.
  Batch 1,500  of  2,110.    Elapsed: 0:10:22.
  Batch 2,000  of  2,110.    Elapsed: 0:13:50.

  Average training loss: 0.41
  Training epcoh took: 0:14:35

Running Validation...
  Accuracy: 0.79
  Validation took: 0:00:29

======== Epoch 3 / 4 ========
Training...
  Batch   500  of  2,110.    Elapsed: 0:03:28.
  Batch 1,000  of  2,110.    Elapsed: 0:06:55.
  Batch 1,500  of  2,110.    Elapsed: 0:10:23.
  Batch 2,000  of  2,110.    Elapsed: 0:13:51.

  Average training loss: 0.34
  Training epcoh t

In [37]:
# 경로 지정
PATH = "/content/drive/MyDrive/DataScience/model/save/steam_model.pt"

In [38]:
def save_checkpoint(epoch, model, optimizer, filename):
    state = {
        'Epoch' : epoch,
        'State_dict' : model.state_dict(),
        'Optimizer' : optimizer.state_dict()
    }
    torch.save(state, filename)

In [39]:
save_checkpoint(epochs, model, optimizer, PATH)

<br>
<br>

# **테스트셋 평가**

In [40]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch   100  of    782.    Elapsed: 0:00:12.
  Batch   200  of    782.    Elapsed: 0:00:25.
  Batch   300  of    782.    Elapsed: 0:00:37.
  Batch   400  of    782.    Elapsed: 0:00:49.
  Batch   500  of    782.    Elapsed: 0:01:01.
  Batch   600  of    782.    Elapsed: 0:01:14.
  Batch   700  of    782.    Elapsed: 0:01:26.

Accuracy: 0.79
Test took: 0:01:36


<br>
<br>

# **새로운 문장 테스트**

In [41]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [42]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [43]:
logits = test_sentences(['연기는 별로지만 재미 하나는 끝내줌!'])

print(logits)
print(np.argmax(logits))

[[-1.8666769  2.0225203]]
1


In [44]:
logits = test_sentences(['주연배우가 아깝다. 총체적 난국...'])

print(logits)
print(np.argmax(logits))

[[ 2.0293608 -2.1927433]]
0


학습한 모델을 가지고 실제 문장을 넣어봤습니다. 출력 로짓은 소프트맥스가 적용되지 않은 상태입니다. argmax로 더 높은 값의 위치를 라벨로 설정하면 됩니다. 0은 부정, 1은 긍정입니다. 위와 같이 새로운 문장에도 잘 분류를 하고 있습니다.
<br>
<br>
<br>

### 영화리뷰 테스트셋

In [45]:
# 네이버 영화리뷰 감정분석 데이터 다운로드
# !git clone https://github.com/e9t/nsmc.git

In [46]:
test = pd.read_csv("/content/drive/MyDrive/DataScience/data/nsmc/ratings_test.txt", sep='\t')
print(test.shape)

(50000, 3)


In [47]:
# 테스트셋의 앞부분 출력
test.head(10)

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
5,7898805,"음악이 주가 된, 최고의 음악영화",1
6,6315043,진정한 쓰레기,0
7,6097171,"마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다",0
8,8932678,갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한c...,0
9,6242223,"이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네..",1


In [48]:
# 리뷰 문장 추출
sentences = test['document']
sentences[:10]

0                                                  굳 ㅋ
1                                 GDNTOPCLASSINTHECLUB
2               뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아
3                     지루하지는 않은데 완전 막장임... 돈주고 보기에는....
4    3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??
5                                   음악이 주가 된, 최고의 음악영화
6                                              진정한 쓰레기
7             마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다
8    갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한c...
9       이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네..
Name: document, dtype: object

In [49]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 굳 ㅋ [SEP]',
 '[CLS] GDNTOPCLASSINTHECLUB [SEP]',
 '[CLS] 뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아 [SEP]',
 '[CLS] 지루하지는 않은데 완전 막장임... 돈주고 보기에는.... [SEP]',
 '[CLS] 3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠?? [SEP]',
 '[CLS] 음악이 주가 된, 최고의 음악영화 [SEP]',
 '[CLS] 진정한 쓰레기 [SEP]',
 '[CLS] 마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다 [SEP]',
 '[CLS] 갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다 [SEP]',
 '[CLS] 이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네.. [SEP]']

In [50]:
# 라벨 추출
labels = test['label'].values
labels

array([1, 0, 0, ..., 0, 0, 0])

In [51]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] 굳 ㅋ [SEP]
['[CLS]', '굳', '[UNK]', '[SEP]']


In [52]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([ 101, 8911,  100,  102,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0])

In [53]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [54]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([ 101, 8911,  100,  102,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0])
tensor(1)
tensor([1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [55]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

### 영화 리뷰 데이터로 시험

In [56]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch   100  of  1,563.    Elapsed: 0:00:12.
  Batch   200  of  1,563.    Elapsed: 0:00:25.
  Batch   300  of  1,563.    Elapsed: 0:00:37.
  Batch   400  of  1,563.    Elapsed: 0:00:49.
  Batch   500  of  1,563.    Elapsed: 0:01:01.
  Batch   600  of  1,563.    Elapsed: 0:01:14.
  Batch   700  of  1,563.    Elapsed: 0:01:26.
  Batch   800  of  1,563.    Elapsed: 0:01:38.
  Batch   900  of  1,563.    Elapsed: 0:01:51.
  Batch 1,000  of  1,563.    Elapsed: 0:02:03.
  Batch 1,100  of  1,563.    Elapsed: 0:02:15.
  Batch 1,200  of  1,563.    Elapsed: 0:02:27.
  Batch 1,300  of  1,563.    Elapsed: 0:02:40.
  Batch 1,400  of  1,563.    Elapsed: 0:02:52.
  Batch 1,500  of  1,563.    Elapsed: 0:03:04.

Accuracy: 0.76
Test took: 0:03:12


### 쇼핑 리뷰 테스트 셋

In [57]:
# 판다스로 데이터 불러오기
df = pd.read_csv("/content/drive/MyDrive/DataScience/data/corpus/sentiment/naver_shopping.txt", sep='\t',
                 names=['label', 'document'])

print(df.shape)

(200000, 2)


In [58]:
# 결측값 확인
print(df.isna().sum())
df.head(3)

label       0
document    0
dtype: int64


,label,document
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...


In [59]:
# 평점을 [1,2] 점은 부정, [4,5] 점은 긍정으로 분류
df['label'] = df['label'].replace(1, 0)
df['label'] = df['label'].replace(2, 0)
df['label'] = df['label'].replace(4, 1)
df['label'] = df['label'].replace(5, 1)

In [60]:
df

,label,document
0,1,배공빠르고 굿
1,0,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,1,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,0,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,1,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ
...,...,...
199995,0,장마라그런가!!! 달지않아요
199996,1,다이슨 케이스 구매했어요 다이슨 슈퍼소닉 드라이기 케이스 구매했어요가격 괜찮고 배송...
199997,1,로드샾에서 사는것보다 세배 저렴하네요 ㅜㅜ 자주이용할께요
199998,1,넘이쁘고 쎄련되보이네요~


In [61]:
df['label'].value_counts(normalize=True)

0    0.500185
1    0.499815
Name: label, dtype: float64

In [62]:
# 훈련셋과 검증셋으로 분리
train, test = train_test_split(df,
                               random_state=3, 
                               test_size=0.25,
                               stratify=df['label'])

print(f"학습용데이터 : {train.shape}\n테스트용데이터 : {test.shape}")

학습용데이터 : (150000, 2)
테스트용데이터 : (50000, 2)


In [63]:
# 데이터별 타겟 분포 확인
print(train['label'].value_counts(normalize=True))
print(test['label'].value_counts(normalize=True))

0    0.500187
1    0.499813
Name: label, dtype: float64
0    0.50018
1    0.49982
Name: label, dtype: float64


In [64]:
# 테스트셋의 앞부분 출력
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)
test.head(10)

,label,document
0,0,이거왜이러는건가요? 네개중사용한두개가이러네요 정품맞나요?
1,0,아~~뒤가 비닐처럼 된거라 전 갠적으로 선호하지 않습니다. 반품도 귀찮아 일단 써보...
2,1,설치할때 망치질하느라 손목이 좀 아프지만 코스트코 고릴라랙보다 예쁘고 튼튼해서 좋습...
3,1,아주 좋아요 야마하
4,0,구성품이 덜오면 안되는 DYI상품인데 망치/ 안전캡 둘다 안온건 너무 했어요
5,0,신발은 참 좋은데 택배 배송문제로 전화 드렸더니 하루종일 안받으시네요 ㅡㅡ 정확히 ...
6,0,생각보다너무하얘서 놀람 저희이불이랑안맞아요
7,0,만족스러워요 만족스러워요 실측사이즈보다 가로세로 100사이즈씩 크게 시켰는데 좋으네...
8,1,선물용으로 샀습니다 배송 좋아요
9,0,맨땅바닥에 떨어뜨린것도아닌데 두껑부분이 깨졌는지 질질세네요 보기만 튼튼해보이지 떨어...


In [65]:
# 리뷰 문장 추출
sentences = test['document']
sentences[:10]

0                      이거왜이러는건가요? 네개중사용한두개가이러네요 정품맞나요?
1    아~~뒤가 비닐처럼 된거라 전 갠적으로 선호하지 않습니다. 반품도 귀찮아 일단 써보...
2    설치할때 망치질하느라 손목이 좀 아프지만 코스트코 고릴라랙보다 예쁘고 튼튼해서 좋습...
3                                           아주 좋아요 야마하
4           구성품이 덜오면 안되는 DYI상품인데 망치/ 안전캡 둘다 안온건 너무 했어요
5    신발은 참 좋은데 택배 배송문제로 전화 드렸더니 하루종일 안받으시네요 ㅡㅡ 정확히 ...
6                              생각보다너무하얘서 놀람 저희이불이랑안맞아요
7    만족스러워요 만족스러워요 실측사이즈보다 가로세로 100사이즈씩 크게 시켰는데 좋으네...
8                                    선물용으로 샀습니다 배송 좋아요
9    맨땅바닥에 떨어뜨린것도아닌데 두껑부분이 깨졌는지 질질세네요 보기만 튼튼해보이지 떨어...
Name: document, dtype: object

In [66]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 이거왜이러는건가요? 네개중사용한두개가이러네요 정품맞나요? [SEP]',
 '[CLS] 아~~뒤가 비닐처럼 된거라 전 갠적으로 선호하지 않습니다. 반품도 귀찮아 일단 써보려구요. [SEP]',
 '[CLS] 설치할때 망치질하느라 손목이 좀 아프지만 코스트코 고릴라랙보다 예쁘고 튼튼해서 좋습니다^^ [SEP]',
 '[CLS] 아주 좋아요 야마하 [SEP]',
 '[CLS] 구성품이 덜오면 안되는 DYI상품인데 망치/ 안전캡 둘다 안온건 너무 했어요 [SEP]',
 '[CLS] 신발은 참 좋은데 택배 배송문제로 전화 드렸더니 하루종일 안받으시네요 ㅡㅡ 정확히 32통 했습니다 문의글남겼더니 전화로 문의 하라네요 ㅡㅡ [SEP]',
 '[CLS] 생각보다너무하얘서 놀람 저희이불이랑안맞아요 [SEP]',
 '[CLS] 만족스러워요 만족스러워요 실측사이즈보다 가로세로 100사이즈씩 크게 시켰는데 좋으네요 생각보다 저렴하게 잘구매했어요 배송도빨라요 [SEP]',
 '[CLS] 선물용으로 샀습니다 배송 좋아요 [SEP]',
 '[CLS] 맨땅바닥에 떨어뜨린것도아닌데 두껑부분이 깨졌는지 질질세네요 보기만 튼튼해보이지 떨어뜨리면 한방이네요 돈아깝게 며칠써보지도못했는데 [SEP]']

In [67]:
# 라벨 추출
labels = test['label'].values
labels

array([0, 0, 1, ..., 0, 0, 0])

In [68]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] 이거왜이러는건가요? 네개중사용한두개가이러네요 정품맞나요? [SEP]
['[CLS]', '이', '##거', '##왜', '##이', '##러', '##는', '##건', '##가', '##요', '?', '네', '##개', '##중', '##사', '##용한', '##두', '##개', '##가', '##이', '##러', '##네', '##요', '정', '##품', '##맞', '##나', '##요', '?', '[SEP]']


In [69]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   9638,  41521, 119164,  10739,  30873,  11018,  71439,
        11287,  48549,    136,   9011,  21789,  41693,  12945,  65219,
       118802,  21789,  11287,  10739,  30873,  77884,  48549,   9670,
        52951, 118913,  16439,  48549,    136,    102,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [70]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [71]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([   101,   9638,  41521, 119164,  10739,  30873,  11018,  71439,  11287,
         48549,    136,   9011,  21789,  41693,  12945,  65219, 118802,  21789,
         11287,  10739,  30873,  77884,  48549,   9670,  52951, 118913,  16439,
         48549,    136,    102,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [72]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

### 쇼핑몰 리뷰 데이터로 시험

In [73]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch   100  of  1,563.    Elapsed: 0:00:12.
  Batch   200  of  1,563.    Elapsed: 0:00:25.
  Batch   300  of  1,563.    Elapsed: 0:00:37.
  Batch   400  of  1,563.    Elapsed: 0:00:49.
  Batch   500  of  1,563.    Elapsed: 0:01:01.
  Batch   600  of  1,563.    Elapsed: 0:01:14.
  Batch   700  of  1,563.    Elapsed: 0:01:26.
  Batch   800  of  1,563.    Elapsed: 0:01:38.
  Batch   900  of  1,563.    Elapsed: 0:01:51.
  Batch 1,000  of  1,563.    Elapsed: 0:02:03.
  Batch 1,100  of  1,563.    Elapsed: 0:02:15.
  Batch 1,200  of  1,563.    Elapsed: 0:02:27.
  Batch 1,300  of  1,563.    Elapsed: 0:02:40.
  Batch 1,400  of  1,563.    Elapsed: 0:02:52.
  Batch 1,500  of  1,563.    Elapsed: 0:03:04.

Accuracy: 0.78
Test took: 0:03:12
